In [1]:
import pickle
import numpy as np
import pandas as pd
from keras.models import model_from_json

In [3]:
def create_dataset( X, time_steps=30):

    Xs = []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
    return np.array(Xs)

def load_model(model_file_path, model_weights_file_path):
    """A function to load the model."""
    global model
    with open(model_file_path, 'r') as f:
        loaded_model_json = f.read()
        model = model_from_json(loaded_model_json)
    model.load_weights(model_weights_file_path)
    return model

In [4]:
def getAnomaliesAutoencoder(df,jsonfile,h5file): 
    filename="transform.pk"
    infile = open(filename,'rb')
    scaler = pickle.load(infile)
    infile.close()
 
    
    data =pd.DataFrame(scaler.transform(df),columns=df.columns,index=df.index)
    
    dt =create_dataset(data)
    
    encoder=load_model(jsonfile,h5file)
    
    prediction= encoder.predict(dt)

    prediction_loss = np.mean(np.abs(prediction - dt), axis=1)
    #sns.distplot(prediction_loss, bins=50, kde=True)
    
    '''************threshold Will get update as model will be fine tuned*********'''
    THRESHOLD = 0.2

    test_score_df = pd.DataFrame(index=df[30:].index,columns={'loss','threshold','anomaly'})
    
    test_score_df["loss"] = prediction_loss
    test_score_df['threshold'] = THRESHOLD
    test_score_df['anomaly'] = test_score_df.loss > test_score_df.threshold
  
    return (test_score_df[test_score_df['anomaly']==True].index).tolist()

In [5]:
new_data=pd.read_csv('AutoEncoder.csv',  index_col='date')
new_data.drop(columns={'region','device'},axis=1,inplace=True)


In [6]:
len(getAnomaliesAutoencoder(new_data,"autoencoder.json","autoencoder.h5"))

408